# Markov Chain basics

Let's demonstrate a simple [Markov Chain](https://en.wikipedia.org/wiki/Markov_chain).
> A Markov chain or Markov process is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event

## US Cities
To demonstrate this, we'll see if we can make something that generates city names.

For this section, we'll get to where we have input data being read successfully.  I downloaded a CSV of city names from https://simplemaps.com/data/us-cities

I've included the first few records of that dataset here, but feel free to replace it with the full data.

In [110]:
import csv
from dataclasses import dataclass
import random
from typing import Iterable, List


In [12]:
def get_cities(input_file: str = 'data/uscities-sample.csv') -> List[str]:
    city_names = []
    with open(input_file, mode='r') as f:
        csv_file = csv.reader(f)
        first = True
        for line in csv_file:
            if first:
                first = False
                continue
            city_names.append(line[0])
    return city_names        

In [13]:
get_cities()

['New York',
 'Los Angeles',
 'Chicago',
 'Miami',
 'Houston',
 'Dallas',
 'Philadelphia',
 'Atlanta',
 'Washington',
 'Boston',
 'Phoenix',
 'Detroit']

Now that we have some input data to work with we're ready to try making a Markov Chain.

## Markov Chain
The idea we're going with is that given some previous letters, generate the next letter.  Or, alternatively, we could stop and say we're done.  Let's add two special characters to our system to signify the end.  We'll use '.' to indicate that the last letter has been generated.  We want to have a method to accept some training data, and a method to generate something.

In [15]:
class MarkovGenerator:
    def __init__(self):
        self._end_char = '.' # Used internally to indicate this is the end of the string.
    
    def train(self, sample: str):
        """Accept one sample of training data."""
        pass

    def generate(self) -> str:
        """Generate one example."""
        pass

What we're going to do is track the probability of the next character based on what we've seen in the training data.  Imagine our full corpus of training data was the word "Catalina".  Let's make the probability of the next letter entirely based on looking just at the previous character.  We'd end up with a table that looks like this:

|Letter | Next letter possibilities |
|-------|---------------------------|
|C | a |
|a | t, l, . |
|t | a |
|l | i |
|n | a |

In our very small training data, if we have a capital C, the only possible next letter is 'a'.  If we have an 'a', then the next letter could be either 't' or 'l', or it could be the end of the sequence.

You can imagine making a similar table that looks back more letters, like 2 or 3.  We're going to extend our class to do these things now.

One trick we're going to use is trim a string down to the last N characters, like this:

In [25]:
'hello'[-2:] # Take the last 2 characters

'lo'

We're going to need to keep track of the information on the right side of that table.  We need to collect statistics about each letter and how often it occurs.

In [47]:
class MarkovCell:
    """Keeps track of statistics for a set of prior letters.
    
    One instance of this corresponds to the **rows** in the "Catalina" example table above.
    """
    def __init__(self, prior_letters: str):
        self.prior_letters = prior_letters
        self.total_count = 0 # How many training examples we have seen.
        self.next_letters = {} # Key: the letter. Value: how many times we have seen that letter.
    
    def add_letter(self, letter: str):
        self.total_count += 1
        count = self.next_letters.get(letter, 0) # Will return 0 if we have never seen this one before.
        count += 1
        self.next_letters[letter] = count
    
    def __repr__(self):
        entries = []
        for key, value in self.next_letters.items():
            entries.append(f'{key}:{value}')
        all_entries = ', '.join(entries)
        return f'MarkovCell({all_entries})'

In [48]:
   
class MarkovGenerator:
    def __init__(self, lookback_size=2):
        self._end_char = '.' # Used internally to indicate this is the end of the string.
        self._lookback_size = lookback_size
        self._stats = {} # This will be keyed by the previous characters. Value will be a MarkovCell instance.
    
    def train(self, sample: str):
        """Accept one sample of training data."""
        prev_chars = ''
        # Add special character to indicate end of sample.
        sample += self._end_char
        for current_char in sample:
            print(f'char={current_char}')

            # If we've never seen this sequence of characters before, add it to our table.
            cell = self._stats.get(prev_chars)
            if not cell:
                cell = MarkovCell(prev_chars)
                self._stats[prev_chars] = cell
            # Update our table with what we've seen for this character.
            cell.add_letter(current_char)

            # Advance the sliding window
            prev_chars += current_char
            prev_chars = prev_chars[-self._lookback_size:] # Keep only the last N characters


    def generate(self) -> str:
        """Generate one example."""
        pass

In [49]:
m = MarkovGenerator()
m.train('Catalina')

char=C
char=a
char=t
char=a
char=l
char=i
char=n
char=a
char=.


In [50]:
m._stats

{'': MarkovCell(C:1),
 'C': MarkovCell(a:1),
 'Ca': MarkovCell(t:1),
 'at': MarkovCell(a:1),
 'ta': MarkovCell(l:1),
 'al': MarkovCell(i:1),
 'li': MarkovCell(n:1),
 'in': MarkovCell(a:1),
 'na': MarkovCell(.:1)}

This looks like what we'd expect looking back two letters.  Let's see if it looks the way we'd expect going back only 1 letter.

In [51]:
m = MarkovGenerator(lookback_size=1)
m.train('Catalina')

char=C
char=a
char=t
char=a
char=l
char=i
char=n
char=a
char=.


In [52]:
m._stats

{'': MarkovCell(C:1),
 'C': MarkovCell(a:1),
 'a': MarkovCell(t:1, l:1, .:1),
 't': MarkovCell(a:1),
 'l': MarkovCell(i:1),
 'i': MarkovCell(n:1),
 'n': MarkovCell(a:1)}

This does look like we want!  Let's try the generate method now.

Imagine we're looking at an 'a' while doing a one character look back, and we need to predict the next letter.  Our starting point (regardless of how many letters we're looking back) is a `MarkovCell`` representing the previous state, along with the possible next step.  Let's pick one up:

In [53]:
cell = m._stats['a']

We can tell how many pieces of training data we've seen for this state:

In [54]:
cell.total_count

3

And we can see what possible next letters could come next:

In [55]:
cell.next_letters

{'t': 1, 'l': 1, '.': 1}

## Making a selection
Given the letters to choose from, we want to make a selection.  Here's how we'll do it.  Imagine a roulette wheel with many spots on it for different possible selections.  We can also imagine that certain possible selections occupy more slots on that wheel.  So imagine we can choose either 'a', 'b', or 'c'.  But 'a' has a weight of 3, and 'b' and 'c' have a weight of one each:

In [56]:
d = {'a': 3, 'b': 1, 'c': 1}

Let's make a roulette wheel function that does this selection for us:

In [75]:
def wheel_select(options: dict, total_count: int):
    """Make a weighted random selection."""
    selected_spot = random.randint(0, total_count)
    current_spot = 0
    for key, slots in options.items():
        if selected_spot <= current_spot + slots:
            return key
        current_spot += slots
    # We should never get here.
    raise Exception('Failed to select an option; total_count must have been wrong')

In [73]:
wheel_select(d, 5)

'a'

That looks good. Let's run it 20 times, and we expect to see 'a' come up more often than the other options. Also let's make sure we see all the options.

In [74]:
[wheel_select(d, 5) for _ in range(20)]

['a',
 'b',
 'a',
 'a',
 'b',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'c',
 'a',
 'b',
 'c',
 'a',
 'c',
 'a']

This looks like good raw materials for our selector.  Although we could put the routine for the weighted random selection into the MarkovCell class, we're going to leave it separate because that's a generally useful function outside of the context of Markov chains (I've used that function many times before in games.)  Let's add a routine to generate the whole text iteratively in the MarkovGenerator.

In [113]:
   
class MarkovGenerator:
    def __init__(self, lookback_size=2):
        self._end_char = '.' # Used internally to indicate this is the end of the string.
        self._lookback_size = lookback_size
        self._stats = {} # This will be keyed by the previous characters. Value will be a MarkovCell instance.
    
    def train(self, sample: str):
        """Accept one sample of training data."""
        prev_chars = ''
        # Add special character to indicate end of sample.
        sample += self._end_char
        for current_char in sample:
            #print(f'char={current_char} prev_chars={prev_chars}')

            # If we've never seen this sequence of characters before, add it to our table.
            cell = self._stats.get(prev_chars)
            if not cell:
                cell = MarkovCell(prev_chars)
                self._stats[prev_chars] = cell
            # Update our table with what we've seen for this character.
            cell.add_letter(current_char)

            # Advance the sliding window
            prev_chars += current_char
            prev_chars = prev_chars[-self._lookback_size:] # Keep only the last N characters


    def generate(self) -> str:
        """Generate one example."""
        generated_text = '' # All the text generated so far.
        window = '' # Last N characters of text generated.
        letter = ''
        while letter != self._end_char:
            cell = self._stats[window]
            letter = wheel_select(cell.next_letters, cell.total_count)
            #print(f'cell={cell} letter={letter}')
            generated_text += letter
            window = generated_text[-self._lookback_size:] # Last N letters of text
        generated_text = generated_text[:-1] # Take off the terminator character.
        return generated_text

    def generate_multi(self, batch_size=20):
        return [self.generate() for _ in range(batch_size)]

Let's see how it goes:

In [107]:
m = MarkovGenerator(lookback_size=1)
m.train('Catalina')

In [108]:
m.generate()

'Calinatatata'

In [109]:
[m.generate() for _ in range(20)]

['Ca',
 'Ca',
 'Ca',
 'Catata',
 'Catatalinatata',
 'Catatalinatatatatatatatatatatalinalinalinatatatalinatatata',
 'Cata',
 'Ca',
 'Cata',
 'Ca',
 'Catalina',
 'Catatatalinatatata',
 'Ca',
 'Calinatalinatalinatalinatata',
 'Cata',
 'Catalinalina',
 'Ca',
 'Ca',
 'Catatatalinatalinatalinatalinalinatalinatata',
 'Catatatata']

This looks promising!  Let's try it with our city names.

In [114]:
def train(input_data: Iterable[str], lookback_size=2) -> MarkovGenerator:
    generator = MarkovGenerator(lookback_size=lookback_size)
    for sample in input_data:
        generator.train(sample)
    return generator

In [115]:
city_generator = train(get_cities())

In [116]:
city_generator.generate_multi()

['Was',
 'New York',
 'Philas',
 'Washingeles',
 'New York',
 'Was',
 'Chingelphicago',
 'Atladelphicago',
 'New York',
 'Chia',
 'Chicago',
 'New York',
 'Miami',
 'Atlanta',
 'New York',
 'Boston',
 'Atlanta',
 'Was',
 'New York',
 'Detroit']

Hey, I think that works!

Now let's try it with training on the full set of US Cities.

You can download the full list from the website at the top of this file, but here's where I have mine:

In [125]:
cities = get_cities('data/uscities-full.csv')

In [126]:
len(cities)

30844

In [127]:
cities[5:10]

['Dallas', 'Philadelphia', 'Atlanta', 'Washington', 'Boston']

In [130]:
city_generator = train(cities)
city_generator.generate_multi()

['Vinhorne',
 'Wastaint',
 'Roseho Gle',
 'Orion th',
 'Mooks',
 'Redingtots',
 'Craver Johne',
 'Parry Har',
 'Lak',
 'Kapen Junden',
 'Ashinna',
 'Oscolon',
 'Royals Cithortrown',
 'Eurton',
 'Mankin',
 'Detterthuryst Heig Garscon',
 'Bakerselwoominville',
 'Jassfords',
 'Belleatt',
 'Elbale']

Let's try it with a lookback of 1:

In [131]:
city_generator = train(cities, lookback_size=1)
city_generator.generate_multi()

['Ceake',
 'Kllurn',
 'Foowoulte',
 'Laigezuee Spillactd',
 'Ce',
 'Ron',
 'Pllie',
 'Lupon',
 'Saron',
 'Norldo',
 'She',
 'Ciche',
 'Crgh Rilleprkthten',
 'Slangrier',
 'Trand Ronddldl-Wison',
 'Nefanddmf Vin-taumongesillouxbetoilld',
 'Do',
 'Wendenncilet',
 'May',
 'Telllal']

And with a lookback of 3:

In [132]:
city_generator = train(cities, lookback_size=3)
city_generator.generate_multi()

['Forest Crookharion',
 'Oakleton',
 'Mont',
 'Campic',
 'Cima',
 'Spoka',
 'Sunrisbury',
 'Reed',
 'Moorhead Lake',
 'Lowrights',
 'Gilma',
 'Delstove',
 'Greenco',
 'Filla Falm Cana',
 'Horse',
 'Freenfield',
 'Wald',
 'Bell',
 'Granjeno',
 'Norwater']

In [133]:
city_generator.generate_multi()

['Menter',
 'Socita Rey',
 'Slico',
 'Ellistown',
 'East Dalez',
 'Nogan',
 'Whittle River',
 'Glendale',
 'Rosedale',
 'House',
 'Hint',
 'Tryopolian Ferryden-on-on-Dumas',
 'Hobson',
 'Jack',
 'Pamester',
 'Burn',
 'Don City Ville',
 'Marshauvin',
 'Pisgah',
 'Hins']

With a lookback of 4, it's quite a bit more coherent:

In [134]:
city_generator = train(cities, lookback_size=4)
city_generator.generate_multi()

['Nesquite',
 'Twin Centra Colony',
 'Necedale Centry Crowhead Brook',
 'Gopher South Ogden',
 'West Petersonville',
 'Kings',
 'Marshall',
 'Springs Lake',
 'Amherst',
 'Moreland',
 'Little',
 'East',
 'Garrelwood',
 'Algodones',
 'Clare',
 'West Costa Springford',
 'Latimera',
 'Minerales',
 'White Meding Water',
 'Jonesburg']

Now let's see what happens if we generate some text based on the Dungeons and dragons handbook.

In [136]:
import codecs
with codecs.open('e:/tmp/dnd_sample.txt', encoding='utf-8') as f:
    dnd_text = f.read()

In [139]:
dnd_text[:100], len(dnd_text)

('The DUNGEONS & DRAGONS game is a roleplaying \r\ngame. In fact, D&D invented the roleplaying game \r\nan',
 27662)

We don't want it to generate a whole book, so let's divide that into sentences.

In [164]:
def preprocess_text(t):
    t = t.replace('\r', '') # Remove carriage returns
    t = t.replace('\n', '') # Remove newlines
    lines = t.split('.')
    return [line.strip() + '.' for line in lines]

In [165]:
dnd = preprocess_text(dnd_text)

In [166]:
len(dnd)

253

In [168]:
dnd[12]

'The adventure is the heart of the D&D game.'

Let's see what we get when we train to 4 characters on the beginning of the D&D handbook:

In [169]:
dnd_generator = train(dnd, lookback_size=4)

In [170]:
dnd_generator.generate_multi()

['For example, and something to repackaged in check rest limitles and your check result purchasm floor your character is 100',
 'Through the or monsters the imagic, and monstealthy',
 'A ROLEPLAY?Your character whole of room for treasure: Dave fun along-awaited by their communitiate to pick up of a speak threative your character “d” follow a past punch sever, or trying rogues, written anything down the game to Playhow the DUNGEONS & DRAGONS & DRAGONS & DRAGONS Basic, and fight covered the Dungeon Masters takes play, you going new miles feate army, even speak into the DUNGEONS game unique is higher is the charget number don’t play a sets to a 10 on the rooms filled adventurer',
 'But if your own every basis or a totally), some by the Dungeon Mastery',
 'During handscape dotted city-states however fabric of each as 0—so appen about strong, and more you by rivers, the sessions, and say in the points as character, character character Ammar, Isidro): “I’ll created, and each adventures, and 

In [171]:
dnd_generator = train(dnd, lookback_size=6)
dnd_generator.generate_multi()

['Each monsters come to a grisly end, torn apart by ferocious monsters of the same group to create a new actions in the players',
 'You can created a story of three',
 '”Dave (DM): “Old stone storytelling a dungeon Tiles: Combat encounters confronting deadly perils',
 'GAME DICEd4 d6 d8 d10 d12 d204E_PHB_Ch01',
 'In 2000, the world is shrouded in a series of adventures and adventure',
 'You creates adventures features: Each place in your way you want to roll initiative game experience as the Dungeon Magazineand Dungeon or stretch out one',
 'You can find some points of adventures',
 '✦ Miniatures, you creature to a world',
 'The Dungeon Master control over the wilderness',
 'When your character, you decides whether you might come in which way your character grows as the variety of location, any time to time, your friends can takes place in your characters, and it was an interesting encounter',
 'The world, and in 1989 the landscape dotted with the other adventures',
 'At sometimes, hol

In [172]:
dnd_generator = train(dnd, lookback_size=7)
dnd_generator.generate_multi()

['',
 'Like the D&D game lets you by asking “What do you do?”, you answer, and the DM create the adventurers need a few “game pieces of all things D&D, a custommade to be used with places between the point, all adventure',
 'This combination, game table',
 'In 1997, Wizards and adventures',
 'The world by storm',
 'Minor kingdoms prosper, to negotiate with its abilities',
 'Adventurers confronting deadly perils',
 'GAME DICEd4 d6 d8 d10 d12 d204E_PHB_Ch01',
 'Your character to do, and the rise of these assumptions',
 'Dave (DM): “You’re by the letter “d” followed by the gong',
 'You can even speak or act in character, you always round down to the Perceptions can employ power to shape spells the mountain',
 'It’s new',
 '',
 'Three Basic Set, and such minor ways: Most characters are a part of the greatest fantastic World The world by storm',
 'This is the heroes would play a single game session or stretch out over many session or stretch out over many sessions of the D&D game: the Dunge